## Testing a Debugging
Python doesn't have compile time static type checking. There's nothing in the interpreter that will ensure that your program will work correctly when you run it. Although a fundamentally dynamic langugage, it has the option for type annotations that can be uses in static analysis to detect many kinds of bugs. 

Test are like an insurance policy on your code. Good tests give you confidence that your code is correct. If you refactor your code, tests verify the *behaviour* not implementation.

### Item 75: Use `repr` Strings for Debugging Output
Print and logs are very useful for debugging code. This issue with printing variables is that the type is not clear. For example:

In [2]:
print(5)
print('5')

5
5


In debugging, these differences matter. Instead of plain print statements, call `repr` on the value before printing. If using the % operator, use '%r' or !r with the f-string.

In [5]:
print(repr(5))
print(repr('5'))
print('%s' %5)


int_value = 5
str_value = '5'
print(f'{int_value!r} != {str_value!r}')

5
'5'
5
5 != '5'


#### `repr` for Object Subclasses
The default implementation of `repr` isn't really helpful. There are two solutions to this:
1. If you have access to the class, define your own `__repr__` special method that returns a string containing the string that recreates the object.
2. If you don't have control over the class definition, access the object's instance dictionary which is stored in the __dict__ attribute.  

In [7]:
class GoodClass:
    def __init__(self, x, y) -> None:
       self.x = x
       self.y = y

    def __repr__(self) -> str:
       return f'GoodClass({self.x!r}, {self.y!r})' 

class OpaqueClass:
    def __init__(self, x, y):
       self.x = x
       self.y = y
    
obj1 = GoodClass(5, '5')
print(obj1)

obj2 = OpaqueClass(5, '5')
print(obj2)
print(obj2.__dict__)

GoodClass(5, '5')
{'x': 5, 'y': '5'}


### Item 76: Verify Related Behaviours in TestCase Subclasses

### Item 77: Isolate Tests from Each Other with `SetUp`, `tearDown`, `setUpModule`, and `tearDownModule`

### Item 78: Use Mocks to Test Code with Complex Dependencies

The behaviour you want to test may have dependencies on other functions. For instance, it might need to call the database to pull a record or uses a helper function. Making a database connection, setting up a schema, populating it with data is difficult and slow. Also, in the case of helper functions and other function calls, those functionalities are not part of the test. 
It is much easier and more robust to mock all these dependencies. A mock lets you provide expected responses for dependent functions given a set of expected calls.
Python has a built in `unittest.mock` module for creating mocks. 
Assume the function `get_animals()` below which queries the database and gets all the animals of a certain `species`. We can mock calls to this function and have those calls return a list of animals:

In [10]:
def get_animals(database, species):
    # Query the database and return a list of (name, last_mealtime)
    pass


from datetime import datetime
from unittest.mock import Mock

mock = Mock(spec=get_animals)
expected_result = [
        ("Spot", datetime(2019, 6, 5, 11, 15)),
        ("Fluffy", datetime(2019, 6, 5, 12, 30)),
        ("Jojo", datetime(2019, 6, 5, 12, 45))
]
mock.return_value = expected_result

The `Mock` class creates a mock function, The `spec` argument indicates that the mock function should act like the given object which is the `get_animals` function and error if it's used in the wrong way. The `return_value` attribute is the value to return when the mock function is called. So we are returning a list of animals whenever `get_animals()` is called without making a database connection! 
Here's an example of calling the wrong attribure on the mock function. Because we pass `get_animals` as the `spec`, only the attribures of this function can be used on the mock object.

In [11]:
mock.blah

AttributeError: Mock object has no attribute 'blah'

We can call the mock function as if we are calling `get_animal()`. For the database argument we can pass a unique object value because the mock function does not actually use it, we just want to make sure that the database argument is passed through any dependent functions that need it. 

In [12]:
database = object()
result = mock(database, 'Meerkat')
assert result == expected_result

This verifies that mock responds as expected. If we also want to verify that the mocked `get_animal` function was called and was called with the correct parameters, we can use the `assert_called_once_with` method. 

In [13]:
mock.assert_called_once_with(database, 'Meerkat')
mock.assert_called_once_with(database, 'Mammal')

AssertionError: expected call not found.
Expected: mock(<object object at 0x1172da660>, 'Mammal')
Actual: mock(<object object at 0x1172da660>, 'Meerkat')

If we don't care about the exact parameters used to make the call, e.g. which database paramater, we can indicate that using the `unittest.mock.ANY` constant. In the snippet below, we call the mock three different times with three differnt database parameters. We then verify that the *most recent call* of the many calls to this function had the second argument "Meerkat".

In [21]:
from unittest.mock import ANY

mock('db1', 'Rabbit')
mock('db2', 'Bison')
mock('db3', 'Meerkat')

mock.assert_called_with(ANY, 'Meerkat')

To mock exceptions being raised you can use `side_effects`:

In [22]:
class CustomError(Exception):
    pass

mock = Mock(spec=get_animals)
mock.side_effect = CustomError('There is an exception!!')
mock('db', 'Meerkat')


CustomError: There is an exception!!

Now let's see how `Mock` can be used in unit tests. Below is a function that does rounds of feeding of zoo animals, given a set of database-interacting functions:

In [23]:
from datetime import datetime, timedelta
from unittest.mock import call, DEFAULT, patch, Mock

class DatabaseConnection:
    pass

def get_animals(database, species):
    # Query the database and return a list of (name, last_mealtime)
    pass

def get_food_period(database, species):
    # Query the database and return the time delta
    pass

def feed_animal(database, name, when):
    # Write to the database
    pass

def do_rounds(database, species, *, utcnow=datetime.utcnow):
    now = utcnow()
    feeding_time_delta = get_food_period(database, species)
    animals = get_animals(database, species)
    fed = 0

    for name, last_mealtime in animals:
        if (now - last_mealtime) > feeding_time_delta:
            feed_animal(database, name, now)
            fed += 1

    return fed


The goal is to verify that when `do_rounds` is called, the correct animal get fed, the correct time of when they were fed is recorded and the number of fed animals is what we expect. For this we need to do a few things:
1. Mock `datetime.utcnow` so the test have stable time that isn't affected by daylight saving time and other similar changes
2. Mock `get_food_period` and `get_animals` to return values that would have come from the database
3. Mock `feed_animal` to accept data that would have been written back to the database

Now the question is, how do you make `do_rounds` accept the mocked functions instead of real functions? One way to do this is inject everything is keyword-only arguments, and pass the mocks as the inputs to overide the results, but this is very verbose and requires changing every function you want to test because you have to make the inputs be keyword-only. A better solution is using `unittest.mock.patch` for injecting mocks. What patch does is that it temporarily reassignes the module or class attributes, e.g. the database accessing functions like `get_animals`. Below we override `get_animals` to be a mock using patch:

In [24]:
from unittest.mock import patch

print('Outside patch: ', get_animals)

with patch('__main__.get_animals'):
    print('Inside patch: ', get_animals)

print('Outside again: ', get_animals)

Outside patch:  <function get_animals at 0x117589ab0>
Inside patch:  <MagicMock name='get_animals' id='4686805696'>
Outside again:  <function get_animals at 0x117589ab0>


`patch` works for many modules, claases and attributes. It can be used in `with` statements, as a function decorator, or in `setUp` and `tearDown` methods of `TestCase` classes.  However, it doesn't work with all calsses such as built-in extensions:

In [26]:
expected_time = datetime(2019, 6, 5, 15, 45)

with patch('datetime.datetime.utcnow'):
    datetime.utcnow.return_value = expected_time

TypeError: cannot set 'utcnow' attribute of immutable type 'datetime.datetime'

There are two solutions to this:
1. Define a helper function to fetch time that can be patched
2. Use a keyword-only argument for the `datetime.utcnow` mock and use a patch for the rest as seen below

We use patch.multiple to create many mocks and set their expectations. The keyword arguments to `patch.multiple` correspond to names in the `__main__` module that we want to override during the test. The `DEFAULT` value indicates that we want a standard `Mock` instance to be created for each name. `autospec=True` makes all the generated mocks to adhere to the specifications of the objects they simulate. 
In the mock context, we see if each of the database accessible functions, `get_animals` and `get_food_period` are called once with the correct parameters and verify that `feed_animal` gets called twice (because of the mocked `last_mealtime` in `get_animal`'s return value) and that the calls are made with the expected inputs in any order (using the `any_order` argument).

But can this code be refactored to be more testable? See Item 79.

In [ ]:
with patch.multiple("__main__",
                    autospec=True,
                    get_animals=DEFAULT,
                    feed_animal=DEFAULT,
                    get_food_period=DEFAULT,
                ):
    now_func = Mock(spec=datetime.utcnow)
    now_func.return_value = datetime(2019, 6, 5, 15, 45)
    
    get_animals.return_value = [
        ("Spot", datetime(2019, 6, 5, 11, 15)),
        ("Fluffy", datetime(2019, 6, 5, 12, 30)),
        ("Jojo", datetime(2019, 6, 5, 12, 45))
    ]
    get_food_period.return_value = timedelta(hours=3)
    
    database = object()
    result = do_rounds(
        database,
        "Meerkat",
        utcnow=now_func
    )
    assert result == 2
    get_food_period.assert_called_once_with(database, "Meerkat")
    get_animals.assert_called_once_with(database, "Meerkat")
    feed_animal.assert_has_calls([
        call(database, "Fluffy", now_func.return_value),
        call(database, "Spot", now_func.return_value),
       
    ],
    any_order=True)

### Item 79: Encapsulate Dependencies to Facilitate Mocking and Testing

The code above requires a lot of boilerplate , which could make it more difficult for new readers to understand what the test is trying to verify.